# MLBD mock exam (spring 2025)

The exam questions are contained in this Jupyter Notebook. The `data` folder contains the data. 

To submit this notebook for the coding questions, you should upload it to Moodle.

Please name your file as "SCIPER_Firstname_Lastname.ipynb".

### Rules

1. In the real exam, you are only allowed to use EPFL's Noto environment, accessible through the link: [https://noto.epfl.ch/](https://noto.epfl.ch/).

2. Please write all your comments in English, and use meaningful variable names in your code. Your code should be clean (e.g., clear names, short functions, no duplicate code, etc.); this is part of your grading.

3. When asked for plots, please include all the needed decorations: namely title, x/y-axis labels, appropriate x/y-ticks, legend, and so on. 

4. We will grade your notebook as is, which means that only the results showed in your evaluated code cells will be considered. Please be sure to submit a **fully-run and evaluated notebook**. If an output cell is not included, it is not graded. We might run the notebook for you if we consider it as necessary for fine-grained checking, so your code should still be runnable properly. Interactive plots, such as those generated using `plotly`, should be **strictly avoided**.

5. The exam is **open book**. You are **allowed** to use all resources on the internet (including Moodle, lecture materials, previous Stackoverflow posts, blogs, GitHub, and your notes) on both the conceptual and coding parts of the exam. However, you are strictly **NOT allowed** to communicate with your peers or outside entities (including any language models, e.g., GPT), post the questions online, or post on this forum during the exam.

6. Please do not cheat. We will report any such occurrence to the legal service of EPFL.


In [11]:
from IPython import display

# YOUR IMPORTS HERE

# Modeling Student Knowledge 

In intelligent learning environments, the selection of problems is based on the predictions of a student model. An accurate representation and prediction of knowledge is hence essential. In this task, you will try two different extensions of the PFA model. 


### The data
The dataset for this task stems from ASSISTments, a tool for assessing math problems and homework. Once they get an assignment, students can complete it at their own pace and with the help of hints and multiple chances. The reduced data set includes problem answers pertaining to three skills. 

The dataset is available in the `data/` directory. Inside the data directory, you will find this file:


#### `answers.csv`: a comma-separated file with the following information about users' answers
- *user_id*: the id of user
- *order_id*: the temporal id (timestamp)
- *skill*: the name of the skill ('Division Fractions', 'Finding Percents', or 'Area Rectangle')
- *correct*: whether the student's answer is correct (1: if the problem's answer is correct at the first attempt, 0: otherwise)
- *ms_first_response*: the number of milliseconds taken to provide the first answer
- *hint_count*: the number of hints requested for that problem
- *attempt_count*: the number of attempts on that problem

## A1) Weight decay (21 points)

You will use the above data to prepare an extended version of the Performance Factor Analysis (PFA) model. A drawback of this model is that it does not **take into account the order of the tasks**, when it counts the numbers of prior successful and failed practices. 

Consider the following example, a user is about to solve a question on a certain skill. Before that, the user has solved four questions involving the same skill; answering correct twice, followed by two incorrect answers. The model therefore counts two prior successes and two prior failures. Another user solves the first two questions incorrectly and then gives two correct answers. The model will again count two prior successes and two prior failures.
It can be seen that the second user seems more confident on that skill, considering his/her last answers and research has indeed shown that recency is important for prediction.

You will therefore extend the original PFA model to deal with these cases. To this end, a decay factor $\sigma$ with $0 < \sigma <= 1$ is needed. Specifically, given the formula originally presented in the course lecture for the original PFA model:

$\pi_{n,i} = \Theta_n + \Sigma_{k} q{i,k} \cdot (\beta_k + \gamma_k \cdot s_{n,k} + \rho_k \cdot f_{n,k})$

**For each interaction of a user on a skill** (that is for every row of the pandas dataframe `answers`, #interactions == #rows in the dataframe), you are asked to replace $\gamma_k$ and $\rho_k$ in the original formula with: 

$\tilde{\gamma_k} = \gamma_k \cdot \sigma^{t-i}$

$\tilde{\rho_k} = \rho_k \cdot \sigma^{t-i}$

where `i` indicates that the current opportunity is the i-th opportunity that the student is solving on that skill, `t` indicates the total number of opportunities of that user with that skill in the data set, and $\sigma$ is the decay factor. 

*Example*: 

Let's assume that a user has experienced `i=3` learning opportunities on the skill `Area Rectangle` and the user has in total `t=4` opportunities on that skill in the data set. 

Let's also assume that the sequence of performances on that skill so far were correct, correct, incorrect `(1,1,0)`, and the decay factor is `0.9`. The decayed_prior_success value of the `i=3` opportunity would be the sum of $1*0.9^3 +1*0.9^2 +0*0.9^1$, while the decayed_prior_failure of the `i=3` opportunity would be the sum of $0*0.9^3 +0*0.9^2 +1*0.9^1$. 


**A1.1) Given the data in the `answers` pandas dataframe, for each skill, use a decay factor $\sigma$ of $0.9$ to train and evaluate a "PFA with decay" model (denoted by A1-PFA in the following).**

Use an appropriate procedure to assess the generalization error of your model.

*This step could take up to 3 minutes of computational time.*

In [23]:
### YOUR CODE HERE ###

**A1.2) Report the mean decayed prior success and the mean decayed prior failure.**  

*No need to discuss here*

In [1]:
### YOUR CODE HERE ###

**A1.3) Using a plot of appropriate type, visually report the performance (use appropriate performance metric(s)) of the A1-PFA model for each skill.**  


In [2]:
### YOUR CODE HERE ###

**A1.4) Justify your choice of performance metric(s) and discuss and interpret your results.**

*YOUR DISCUSSION HERE*

## A2) Extra information: Time Spent on First Response (17 points)

In A1 you developed an extension from PFA in order to be able to take the order of tasks into account. In this task (A2) you will develop a different extension starting from the original PFA model. 

A second drawback of the original PFA model is that it disregards how long students take to answer the questions. For example, if a student got a question correct after 15 seconds and another student required 5 minutes to get the same question correct, the model will make the same predictions. However, it is likely that the fast student already knows the skill better than the slow and inconfident student. Therefore, in a second extension of PFA (later denoted by A2-PFA), you will extend the original PFA model to be able to differentiate between fast and slow students when making predictions.

To do so, you will need to look at the time students spent on their first response (`ms_first_response`). For each user and skill, you must determine if the user has a response time above or below average in comparison to the other users.

*Example*:

Imagine that for skill `Area Rectangle`, `Alice` solved three questions and the response time was 300, 900, and 600. Thus, the average response time for student `Alice` is 600 ms. Let's assume there are only two more students (`Bob`, `Charlie`)  with an average response time of 200 and 700 respectively. Thus, the average response time per user for the skill `Area Rectangle` is 500 ms. 

In this case, `Bob` is *faster* than the average, and `Alice` and `Charlie` are *slower*. 


**A2.1) Extend the original PFA model to be able to represent fast and slow users differently. A user is defined as slow (for a skill) if their mean response time is higher than the average mean response time of that skill; a user is defined as fast (for a skill) if their mean response time is lower or equal to the average mean response time for that skill.** 



In [32]:
### YOUR CODE HERE ###

**A2.2) Report the percentage of fast students per skill.**

*No need to discuss here*

In [3]:
### YOUR CODE HERE ###

**A2.3) Using a plot of appropriate type, visually report the performance (use appropriate performance metric(s)) of the A2-PFA model for each skill.**   

In [4]:
### YOUR CODE HERE ###

**A2.4) Explain and justify how you adapted the equation of the original PFA model to obtain the A2-PFA model (able to model fast and slow students differently). Discuss and interpret your results.**

*YOUR DISCUSSION HERE*

##  A3) Compare your models (12 points)

How do the model A1-PFA and A2-PFA models compare to the original PFA?

**A3.1) Provide a visualization comparing the performance metric(s) of the three models per skill for A1-PFA1, A2-PFA and the original PFA model (as seen in class).**


In [5]:
### YOUR CODE HERE ###

**A3.2) Discuss and interpret your results.** 

*YOUR DISCUSSION HERE*